In [1]:
import sys
sys.path.append('I:/YaoTony/code/x8313')
sys.path.append('I:/YaoTony/saleem tasks')
import Harville
#ys.path.append('/Users/saleemhuda/Dropbox/X8313_Live')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd
import os.path

In [ ]:
# dynamic strategy combination + scoring

import os
from pandas import DataFrame, concat, MultiIndex, pivot_table
from horse.betsim.wrap.daily import DailyRaces
from horse.betsim.wrap.jcapper import JCapper
from horse.betsim.wrap.bets import Bets
from datetime import datetime, timedelta
from experiments.fairvalue.models.history import map_jcp_itsp
from experiments.fairvalue.models.strategy import get_combinations_by_bet_type, generate_wagers, ProbModel
from experiments.fairvalue.models.probability import compute_probs_races,  add_win_probs_from_score
from experiments.fairvalue.models.selector import SelectMinimumRunners
from experiments.fairvalue.models.strategy import Strategy, ProbModel
from horse.betsim.data.db.engines import engine_ac
from experiments.fairvalue.models.history import History, LabelHistory
from experiments.fairvalue.models.probability import generate_permutation_matrix, HarvilleTri , MultiProbability, multiprob_base, multiprob_harville
#from experiments.fairvalue.models.weight import
import scipy.stats as ss
import horse

import itertools
import pandas as pd
import os

# SCRIPT GLOBALS

# output path
horse_path = os.path.dirname(horse.__file__)
path_project = os.path.dirname(horse_path)
path_scratch = os.path.join(path_project, 'scratch')
path_live = os.path.join('/Users/saleemhuda/Dropbox/X8313_LIVE/')
# target date
today = datetime.today()
yesterday = today - timedelta(days=1)

history = History(engine_ac, verbose=True)
# get sim dates from history.match_wom_dates(today)
#dates_history = history.match_wom_dates(today, start_year=2015, end_year=2016, filter_month=True)
dates_history = [date(2016,7,3)]
history.load(dates_history)
# add computed columns and factors
target_date_format = today.strftime("%Y%m%d")
hist_date_format= dates_history[0].strftime("%Y%m%d")
from horse.betsim import data
import os.path
track_detail = os.path.join(data.__path__._path[0], 'track_detail.csv')
df_track_detail = pd.read_csv(track_detail, index_col=['x8_track_sym'])
map_jcp_itsp = df_track_detail.set_index('jcp_track_sym')['itsp_track_sym'].to_dict()
map_rebates_win = df_track_detail.WN.to_dict()

lbh = LabelHistory()
lbh.load(history)

dfX_hist = lbh.dfX.copy()

In [3]:
dfX_hist = dfX_hist[dfX_hist.x8_num_starters_pp>5].copy()
dfX_hist['payout_win'] = dfX_hist['final_tote_odds']+1
dfX_hist['rebate_win'] = dfX_hist['track_id'].map(lambda x:map_rebates_win.get(x,0.0))
dfX_hist.set_index(['race_id','runner_id','x8name_runner'], inplace=True)

In [4]:

dfX_hist_probs = dfX_hist[[c for c in dfX_hist.columns if c.find('prob')>-1]].copy()
dfX_hist_winners = dfX_hist[dfX_hist.official_finish_position==1].copy()
dfX_hist_payoutpools = lbh.df_post_payout_pool.set_index(['track_id','date','race_id','bet_type'])

#[['HDWSpeedParForClassLevel','nonbetting','days_since_last_race','distance','track_id','post_position','coupled_entry','full_race_conditions','race_classification','race_classification_0','purse_0','purse_1', 'finish_pos_0', 'finish_pos_1', 'riderROI','trainerROI', 'final_tote_odds', 'x8_probs_odds_norm_win_final', 'jockey','trainer','race_classification_0', 'x8runner_HDWPSRRating_norm_par','prob_x8runner_HDWPSRRating_norm_par', 'x8ratio_earnings_0_claim']].copy()
dfY = dfX_hist[['x8is_win', 'x8is_exacta', 'x8is_trifecta', 'x8is_superfecta','final_tote_odds','official_finish_position']].copy()


dfX_hist.reset_index(inplace=True)





#get good trainers
lbh.get_trainer_wins(attr_result='x8is_win')
lbh.get_jockey_wins(attr_result = 'x8is_win')
lbh.get_trainerjockey_wins(attr_result='x8is_win')

winner_trainer_count = dfX_hist_winners.trainer.value_counts()
winner_jockey_count = dfX_hist_winners.jockey.value_counts()
top_jockeys  =winner_jockey_count[winner_jockey_count>2].index.values
top_trainers  =winner_trainer_count[winner_trainer_count>2].index.values

dfX_trainer_pattern = dfX_hist_winners[dfX_hist_winners.trainer.isin(top_trainers)][['trainer','race_classification','race_classification_0']]


def _get_calibration_data(lbh, tracks=None):
    """
    Puts payout , pool data together with runner probabiility data
    Needs probabilities, final_tote_odds, poolsize, payout, runners
    :return:
    :rtype:
    """
    # merge runner factor data in dfX with payout data
    # then merge pool data
    dfmergepaydfx = pd.merge(lbh.df_pivot_payout.reset_index(), lbh.dfX, left_on=['race_id'], right_on=['race_id'],
                             how='right')
    df_calibrate = pd.merge(dfmergepaydfx, lbh.df_pivot_pool.reset_index(), left_on=['race_id'],
                                 right_on=['race_id'], how='right', suffixes=('_payout','_pool'))
    return df_calibrate

# weighting constants and cutoff for wager df
strat_dynamic_weights = (1,1,1) # TODO make function
#strat_threshold = 1.7 # TODO make function
#dfX_hist_probs = dfX_hist.groupby('race_id').apply(lambda df: add_win_probs_from_score(df,'x8runner_HDWPSRRating_norm_par')).reset_index()

#target_raceid_pre = 'PID_20180618_6'






dfX_hist_probs_TR_HDW = compute_probs_races(races=lbh.races, bet_type='TR', attr_win_prob='prob_x8runner_HDWPSRRating_norm_par',
                            multiprob_model=multiprob_base)
dr_pre = DailyRaces(state='pre', verbose=True)
dr_pre.load([today])
dr_pre.add_computed_columns()

dfX_today_pre = dr_pre.df.copy().set_index(['race_id_pp','runner_id','x8name_pp'])
dfX_today_trainers = dfX_today_pre[dfX_today_pre.trainer_pp.isin(top_trainers)]
dfX_today_jockeys = dfX_today_pre[dfX_today_pre.jockey_pp.isin(top_jockeys)]

#TODO: This needs to map to one attr  in race




#track_id
list_tracks_hist = dfX_hist['track_id']
list_tracks_today = dfX_today_pre['jcp_track_sym_pp']


#dfX_today_pre = add_win_probs_from_score(dfX_today_pre,'x8runner_HDWPSRRating_norm_par')
dfX_probs = dfX_today_pre.groupby('race_id_pp').apply(lambda df: add_win_probs_from_score(df,'x8runner_HDWPSRRating_norm_par'))



attr_model = 'prob_x8runner_HDWPSRRating_norm_par'
def compute_simple_payout(df, attr_model, ascending=False, bet_amount=1.0):
    """
    Add columns for quick calculation of Win bets % payout
    :param df: Dataframe from dataset for multiple races
    :param attr_model: (string)an attribute / prob or score in the dataframe that can be ranked
    :param ascending: False if best is higher score i.e. probs
    :param bet_amount: default to 1.0
    :return: Dataframe with columns added

    """
    #df['is_win'] = df['official_finish_position'].map(lambda x:int(x==1)) #1 if this runner is winner
    #add rank attribute for model
    df['rank_' + attr_model] = df.groupby('race_id_pp')[attr_model].transform(lambda x:x.rank(ascending=ascending))
    df['bet_amount'] = df['rank_' + attr_model].map(lambda x:int(x<1.5) * bet_amount) #we bet 1 unit if rank is 1
    df['is_wager'] = df['bet_amount'].map(lambda x:int(x>0)) #indicator for rows with bets
    df['is_win'] = df['official_finish_position'].map(lambda x:int(x==1))
    df['is_paid'] = df['is_wager'] * df['is_win']
    #This is the payout (without impact) of the
    df['payout'] = df['is_win']*df['bet_amount']*df['payout_win'].fillna(0.0) + df['bet_amount']*df['rebate_win']#rebate
    return df
from functools import wraps

class VerticalWagers():
    """
    A class for generating WN, PL, SH, EX, TR, SU bets from a probability model
    Needs:
    input_args:
    prob_model: returns WN probs for each runner
    permute_model : returns multihorse probs: ie. HarvilleTR
    pool_sizer: function which returns a dict defaults to: {'WN': 10000, 'EX':10000, 'TR':10000, 'SU': 10000}
    risk_policy: function which returns scaled bets

    outputs:
    df_bets: a dataframe with TicketBase() attributes




    """
    def __init__(self, multiprob_model):
        self.multiprob_model = multiprob_model






df_probs_TR_HDW_harville = compute_probs_races(races=dr_pre.races, bet_type='TR', attr_win_prob='probs_morning_line_pp',
                            multiprob_model=multiprob_harville)

def dump_target_tracks():

    dfX_probs.to_csv(path_live+'df_probs_{}.csv'.format(target_date_format), index_label=dfX_probs.index.names)
    print("wrote df_probs_{}.csv".format(target_date_format))
    dfX_hist_probs.to_csv(path_live + 'df_hist_probs_{}.csv'.format(hist_date_format), index_label=dfX_hist_probs.index.names)
    print("wrote df_hist_probs_{}.csv".format(hist_date_format))
    dfX_hist_payoutpools.to_csv(path_live + 'df_hist_probs_{}.csv'.format(hist_date_format), index_label=dfX_hist_payoutpools.index.names)
    print("wrote df_hist_payoutpools_{}.csv".format(hist_date_format))
    return True

dump_target_tracks()


# Functions for adding arbitrary sets of filtering criteria
def merge_dicts(*dict_args):
    """
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    """
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result


def apply_criteria(df, dict_crit):
    # apply dicts in dict_crit
    # Applies criteria to filter runners in df'
    # i.e.'x8is_par_HDWPSRRating': lambda df:df[df['x8is_HDWPSRRating_norm_par']>0]

    for crit_name, crit_func in dict_crit.items():
        print("applying crit {}".format(crit_name))
        print("num_runners before: {}".format(len(df.name_pp.unique())))  # TODO: add race count, track count
        df = crit_func(df)
        print("num_runners after: {}".format(len(df.name_pp.unique())))
    return df


def compute_filtered_probs(df, transform_dict=None):
    df['sum_prob_par'] = df.groupby('race_id_pp')[
        'prob_x8runner_HDWPSRRating_norm_par'].transform(lambda x: x.sum())
    df['max_prob_HDW'] = df.groupby('race_id_pp')[
        'prob_x8runner_HDWPSRRating_norm_par'].transform(lambda x: x.max())
    df['entropy_filtered_prob'] = df.groupby('race_id_pp')[
        'prob_x8runner_HDWPSRRating_norm_par'].transform(lambda x: ss.entropy(x, base=len(x))).fillna(0.0)
    return df






def get_rebates_by_track_id(df, default_rebate=0.0):
    """
    can use track_detail.csv

    """

    pass



def compute_advantage(dfpay, rebate=0.0):
    # Computes the pct advantage as:
    # Win% - (Loss% / (mean_odds + rebate))
    # dfpay: comes from compute_simple_payout()
    # rebate
    pct_win = dfpay.groupby('race_id')['is_paid'].sum().value_counts(normalize=True)[1]

    pct_loss = 1.0 - pct_win
    mean_odds = (dfpay[dfpay.is_paid > 0]['final_tote_odds'] + dfpay[dfpay.is_paid > 0]['rebate_win']).mean()
    advantage = pct_win - pct_loss / (mean_odds + rebate)
    print(advantage)
    return advantage

#Here we create dictionaries for the criteria we can apply using information available before betting starts

dict_par_track = dfX_hist_winners.groupby(['jcp_track_sym_pp','race_type','distance'])['x8runner_HDWPSRRating_norm_par'].mean()


#Filter tracks to bettable tracks

dict_criteria_numstarters = {'x8_num_starters_pp': lambda df:df[df['x8_num_starters_pp']>5]}

crit_is_par = {'x8is_par': lambda df: df[df['x8_is_HDWPSRRating_norm_par']>0],
                            'x8is_top4_ml': lambda df:df[df['rank_morning_line_pp']<5]}


for race probs_TR_prob_x8runner_HDWPSRRating_norm_par  probs GIL_20160703_8 sum to 0.999999999999997
                                                    probs_TR_prob_x8runner_HDWPSRRating_norm_par
(GIL_20160703_8_4, GIL_20160703_8_3, GIL_201607...                                      0.002976
(GIL_20160703_8_1, GIL_20160703_8_2, GIL_201607...                                      0.002976
(GIL_20160703_8_2, GIL_20160703_8_4, GIL_201607...                                      0.002976
(GIL_20160703_8_8, GIL_20160703_8_4, GIL_201607...                                      0.002976
(GIL_20160703_8_2, GIL_20160703_8_8, GIL_201607...                                      0.002976
(GIL_20160703_8_3, GIL_20160703_8_5, GIL_201607...                                      0.002976
(GIL_20160703_8_6, GIL_20160703_8_8, GIL_201607...                                      0.002976
(GIL_20160703_8_4, GIL_20160703_8_6, GIL_201607...                                      0.002976
(GIL_20160703_8_8, GIL_201

loading gts and pp..
pp.load(2018/06/30) loading 36 race cards..
merging dataframes..
loading 283 Race() objects..
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par


/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_n

added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_n

In [5]:
def _get_calibration_data(_lbh, tracks=None):
    """
    Puts payout , pool data together with runner probabiility data
    Needs probabilities, final_tote_odds, poolsize, payout, runners
    :return:
    :rtype:
    """
    # merge runner factor data in dfX with payout data
    # then merge pool data
    dfmergepaydfx = pd.merge(_lbh.df_pivot_payout.reset_index(), _lbh.dfX, left_on=['race_id'], right_on=['race_id'],
                             how='right')
    df_calibrate = pd.merge(dfmergepaydfx, _lbh.df_pivot_pool.reset_index(), left_on=['race_id'],
                                 right_on=['race_id'], how='right', suffixes=('_payout','_pool'))
    return df_calibrate


In [6]:
df_CALIBRATE = _get_calibration_data(lbh)

In [15]:
df_CALIBRATE.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1402 entries, 0 to 1401
Data columns (total 265 columns):
race_id                                1402 non-null object
EX_payout                              1383 non-null float64
PL_payout                              1383 non-null float64
SH_payout                              1383 non-null float64
SU_payout                              1273 non-null float64
TR_payout                              1371 non-null float64
WN_payout                              1383 non-null float64
index                                  1383 non-null float64
track_id                               1383 non-null object
date                                   1383 non-null object
race_number                            1383 non-null float64
distance                               1383 non-null float64
approx_dist                            1383 non-null object
surface                                1383 non-null object
race_type                              1383

In [16]:
dfX_payoutpool = lbh.df_post_payout_pool.copy()


In [18]:
dfX_payoutpool.to_csv(path_live+'df_calibrate_payoutpool_2080630.csv',index=False )

In [17]:
dfX_payoutpool.head()


,race_id,track_id,date,race_number,distance,approx_dist,surface,race_type,sex_restriction,age_restriction,...,runners,race_num,bet_amount,num_exacta,num_trifecta,num_superfecta,pct_cover_exacta,pct_cover_trifecta,pct_cover_superfecta,is_cover_tri
0,ABT_20160703_1,ABT,20160703,1,440.0,False,T,N,N,H,...,6/ALL/ALL,1,1.0,56,336,1680,0.116071,0.019345,0.003869,0
1,ABT_20160703_1,ABT,20160703,1,440.0,False,T,N,N,H,...,6/9,1,1.0,56,336,1680,1.313393,0.218899,0.043780,0
2,ABT_20160703_1,ABT,20160703,1,440.0,False,T,N,N,H,...,6,1,1.0,56,336,1680,0.147321,0.024554,0.004911,0
3,ABT_20160703_1,ABT,20160703,1,440.0,False,T,N,N,H,...,6,1,1.0,56,336,1680,0.147321,0.024554,0.004911,0
4,ABT_20160703_1,ABT,20160703,1,440.0,False,T,N,N,H,...,6,1,1.0,56,336,1680,0.147321,0.024554,0.004911,0


In [7]:
df_CALIBRATE.official_finish_position

0        1.0
1        2.0
2        3.0
3        4.0
4        5.0
5        6.0
6        7.0
7        8.0
8        9.0
9        1.0
10       2.0
11       3.0
12       4.0
13       5.0
14       6.0
15       7.0
16       1.0
17       2.0
18       3.0
19       4.0
20       5.0
21       NaN
22       NaN
23       1.0
24       2.0
25       3.0
26       4.0
27       5.0
28       6.0
29       NaN
        ... 
1372     1.0
1373     2.0
1374     3.0
1375     4.0
1376     5.0
1377     6.0
1378     7.0
1379     8.0
1380     9.0
1381    10.0
1382    11.0
1383     NaN
1384     NaN
1385     NaN
1386     NaN
1387     NaN
1388     NaN
1389     NaN
1390     NaN
1391     NaN
1392     NaN
1393     NaN
1394     NaN
1395     NaN
1396     NaN
1397     NaN
1398     NaN
1399     NaN
1400     NaN
1401     NaN
Name: official_finish_position, Length: 1402, dtype: float64

In [8]:
df_CALIBRATE.head()

,race_id,EX_payout,PL_payout,SH_payout,SU_payout,TR_payout,WN_payout,index,track_id,date,...,year,month,weekday,prob_x8runner_HDWPSRRating_norm_par,EX_pool,PL_pool,SH_pool,SU_pool,TR_pool,WN_pool
0,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,98.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.137261,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
1,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,99.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.083753,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
2,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,100.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.471878,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
3,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,101.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.137261,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
4,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,102.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.000223,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0


In [ ]:
#df_CALIBRATE = pd.read_csv('./DF_CALIBRATE_20180629.csv')
#df_CALIBRATE.head()

In [9]:
df = df_CALIBRATE[['race_id','EX_payout','runner_program_number_pp','x8is_exacta','official_finish_position','prob_x8runner_HDWPSRRating_norm_par']]
df = df.sort_values(by=['race_id','official_finish_position']).dropna(subset=['EX_payout'])
df.head()

,race_id,EX_payout,runner_program_number_pp,x8is_exacta,official_finish_position,prob_x8runner_HDWPSRRating_norm_par
0,ALB_20160703_10,13.3,2,1.0,1.0,0.137261
1,ALB_20160703_10,13.3,1,1.0,2.0,0.083753
2,ALB_20160703_10,13.3,8,0.0,3.0,0.471878
3,ALB_20160703_10,13.3,7,0.0,4.0,0.137261
4,ALB_20160703_10,13.3,3,0.0,5.0,0.000223


In [ ]:
df.to_csv('DF_CALIBRATE_20180630.csv', index=False)

In [10]:
#exacta bet type
import numpy as np
df_exacta = df[np.isfinite(df['prob_x8runner_HDWPSRRating_norm_par'])]#delete rows when race only has one runner
df_exacta['is_paid']=0
race_set = set(df_exacta['race_id'])
for race in race_set:
    df_exacta_sub = df_exacta[df_exacta['race_id']==race]
    #fav is prob ranking 1 of exacta combination and its prob
    fav = Harville.harville_multiprob(list(df_exacta_sub['runner_program_number_pp']),list(df_exacta_sub['prob_x8runner_HDWPSRRating_norm_par']),'Exacta')[0]
    df_exacta_sub_ex = df_exacta_sub[df_exacta_sub['x8is_exacta']>0]#extract extacta runners
    df_exacta_sub_ex=df_exacta_sub_ex.reset_index(drop=True)
    #suppose is_paid is 0 if only one runner participate the race
    if len(df_exacta_sub_ex)>1:
        if df_exacta_sub_ex.loc[1,'official_finish_position']==1:
            if set(fav[0:2]).issubset(set(df_exacta_sub_ex['runner_program_number_pp'])):
                df_exacta.loc[df_exacta['race_id']==race,'is_paid']=1
        elif list(df_exacta_sub_ex['runner_program_number_pp'])==fav[0:2]:
            df_exacta.loc[df_exacta['race_id']==race,'is_paid']=1


/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
df_exacta[df_exacta['is_paid']==1]

,race_id,EX_payout,runner_program_number_pp,x8is_exacta,official_finish_position,prob_x8runner_HDWPSRRating_norm_par,is_paid
85,APX_20160703_7,11.2,1,1.0,1.0,0.191468,1
86,APX_20160703_7,11.2,8,1.0,2.0,0.172299,1
87,APX_20160703_7,11.2,4,0.0,3.0,0.123758,1
88,APX_20160703_7,11.2,5,0.0,4.0,0.098022,1
89,APX_20160703_7,11.2,10,0.0,5.0,0.000030,1
90,APX_20160703_7,11.2,2,NaN,NaN,0.086895,1
91,APX_20160703_7,11.2,3,NaN,NaN,0.123758,1
92,APX_20160703_7,11.2,6,NaN,NaN,0.029978,1
93,APX_20160703_7,11.2,7,NaN,NaN,0.086895,1
94,APX_20160703_7,11.2,9,NaN,NaN,0.086895,1


In [12]:
#exacta box bet type, Quinella
import numpy as np

df_quinella = df[np.isfinite(df['prob_x8runner_HDWPSRRating_norm_par'])]#delete rows when race only has one runner
df_quinella['is_paid']=0
race_set = set(df_quinella['race_id'])
for race in race_set:
    df_quinella_sub = df_quinella[df_quinella['race_id']==race]
    #fav is prob ranking 1 of box combination and its prob
    fav = Harville.harville_box(list(df_quinella_sub['runner_program_number_pp']),list(df_quinella_sub['prob_x8runner_HDWPSRRating_norm_par']),'Exacta')[0]
    # we can also use fav = Harville.harville_multiprob(list(df_quinella_sub['runner_program_number_pp']),list(df_quinella_sub['prob_x8runner_HDWPSRRating_norm_par']),'Quinella')[0]
    
    df_quinella_sub_ex = df_quinella_sub[df_quinella_sub['x8is_exacta']>0]#extract extacta runners
    if set(fav[0:2]).issubset(set(df_quinella_sub_ex['runner_program_number_pp'])):
        df_quinella.loc[df_quinella['race_id']==race,'is_paid']=1

/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
df_quinella['is_paid'].value_counts()

0    1153
1     228
Name: is_paid, dtype: int64

In [ ]:
df_WOX_10  = df_CALIBRATE.groupby('race_id').get_group('WOX_20160703_10')

In [ ]:
df = df_CALIBRATE.set_index(['race_id','official_finish_position','trainer','jockey'])



In [ ]:
dfWOX10 = df.groupby(['race_id']).get_group('WOX_20160703_10')